In [1]:
# archivo para prácticas 

import pandas as pd

data=pd.read_csv('./clinical_analytics.csv')

In [2]:
data.head()

,Admit Source,Admit Type,Appt Start Time,Care Score,Check-In Time,Clinic Name,Department,Diagnosis Primary,Discharge Datetime new,Encounter Number,Encounter Status,Number of Records,Wait Time Min
0,Emergency Room,Emergency,2014-01-02 11:38:50 PM,2,2014-01-02 11:24:00 PM,Madison Center,Cardiology,NaN,NaN,P7P4KC587,Cancelled,1,14
1,Emergency Room,Emergency,2014-01-02 11:44:18 PM,2,2014-01-02 11:24:00 PM,Madison Center,Cardiology,NaN,NaN,P7P4KC587,Cancelled,1,20
2,Emergency Room,Emergency,2014-01-02 11:47:11 PM,2,2014-01-02 11:24:00 PM,Madison Center,Cardiology,NaN,NaN,P7P4KC587,Cancelled,1,23
3,Emergency Room,Emergency,2014-01-08 10:38:04 PM,4,2014-01-08 9:00:00 PM,Madison Center,Emergency,NaN,NaN,PK559C587,Cancelled,1,98
4,Emergency Room,Emergency,2014-01-09 12:00:26 AM,4,2014-01-08 10:47:00 PM,Madison Center,Emergency,NaN,NaN,48559C587,Cancelled,1,73


In [3]:
data.columns

Index(['Admit Source', 'Admit Type', 'Appt Start Time', 'Care Score',
       'Check-In Time', 'Clinic Name', 'Department', 'Diagnosis Primary',
       'Discharge Datetime new', 'Encounter Number', 'Encounter Status',
       'Number of Records', 'Wait Time Min'],
      dtype='object')

In [8]:
# Preparación de los datos
import pandas as pd
df=data
df['Appt Start Time'] = pd.to_datetime(df['Appt Start Time'])
df['Day of Week'] = df['Appt Start Time'].dt.day_name()
df['Hour of Day'] = df['Appt Start Time'].dt.hour

heatmap_data = df.groupby(['Day of Week', 'Hour of Day']).agg(
    Patient_Count=pd.NamedAgg(column='Number of Records', aggfunc='sum'),
    Avg_Care_Score=pd.NamedAgg(column='Care Score', aggfunc='mean')
).reset_index()

heatmap_count = heatmap_data.pivot(index="Day of Week", columns="Hour of Day", values="Patient_Count")
heatmap_score = heatmap_data.pivot(index="Day of Week", columns="Hour of Day", values="Avg_Care_Score")


# Crear los textos para los tooltips
tooltip_texts = [
    [
        f"<br>Calificación Promedio: {heatmap_score.iloc[i, j]:.2f}"
        if not pd.isna(heatmap_score.iloc[i, j]) else "No data"
        for j in range(len(heatmap_count.columns))
    ]
    for i in range(len(heatmap_count.index))
]


In [10]:
# Orden explícito para los días de la semana
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Reordenar el DataFrame por días de la semana
heatmap_count = heatmap_count.reindex(days_order)

# Reordenar manualmente la lista tooltip_texts según days_order
# Asumiendo que tooltip_texts es una lista de listas donde cada sublista corresponde a un día
ordered_tooltip_texts = [None] * len(days_order)
current_days = list(heatmap_count.index)

for i, day in enumerate(days_order):
    index = current_days.index(day)
    ordered_tooltip_texts[i] = tooltip_texts[index]

# Ahora crea el mapa de calor con el orden correcto
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
    z=heatmap_count.values,
    x=heatmap_count.columns,
    y=heatmap_count.index,
    hoverongaps=False,
    colorscale='Viridis',
    hoverinfo='text',
    text=ordered_tooltip_texts  # Usar la lista reordenada
))

for i, day in enumerate(heatmap_count.index):
    for j, hour in enumerate(heatmap_count.columns):
        fig.add_annotation(
            x=hour,
            y=day,
            text=str(heatmap_count.at[day, hour]),
            showarrow=False,
            font=dict(color="white")
        )

# Añadir título y etiquetas
fig.update_layout(
    title='Volumen de Pacientes por Día y Hora',
    xaxis_title='Hora del Día',
    yaxis_title='Día de la Semana'
)

# Mostrar la figura
fig.show()
